#  Controlled SharePoint Library Hacker 

**Benjamin Corbett**  
April 23, 2025

## Import Libraries

In [1]:
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
from abc import ABC, abstractmethod
import smtplib
import json
from itertools import zip_longest
from typing import List


load_dotenv() 

True

## Abstract Base Class

In [4]:
class Periodical(ABC):
    def __init__(self, title, author, available=True):
        self.title = title
        self.author = author
        self.available = available
        
    @abstractmethod
    def display_info(self):
        pass

## Subclasses

In [7]:
class Book(Periodical):
       
    def display_info(self):
        print(f"[Book] '{self.title} by {self.author}'")
        print(f"Available: {'Yes' if self.available else 'No'}")

In [9]:
class Newspaper(Periodical):
    def display_info(self):
        print(f"[Newspaper] '{self.title} by {self.author}'")
        print(f"Available: {'Yes' if self.available else 'No'}")

In [11]:
class Magazine(Periodical):
    def display_info(self):
        print(f"[Magazine] '{self.title} by {self.author}'")
        print(f"Available: {'Yes' if self.available else 'No'}")

In [13]:
class Library:
    def __init__(self):
        self.books: List[Periodical] = []

    def add_book(self, book: Periodical):
        self.books.append(book)
        print(f"Added {book.title} to my library")

    def remove_book(self, title: str):
        for book in self.books:
            if book.title == title:
                self.books.remove(book)
                print(f"Removed {book.title} from my library")
                return
        print("Not found.")

    def list_all(self) -> str:
        if not self.books:
            return "Your library is empty."

        result = ["📚 Library:"]
        for book in self.books:
            periodical_type = book.__class__.__name__
            result.append(f"[{periodical_type}] Title: {book.title}")
            result.append(f"Author: {book.author}")
            result.append(f"Available: {'Yes' if book.available else 'No'}")
        return '\n'.join(result)

    def to_json(self) -> str:
        """Returns the library as a JSON string"""
        data = [
            {
                "Title": book.title,
                "Author": book.author,
                "Available": "Yes" if book.available else "No"
            }
            for book in self.books
        ]
        return json.dumps(data, indent=2)  
       

## Book Scraping

In [16]:
url = "https://books.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [17]:
category_elements = soup.select('.side_categories ul li ul li a')

In [20]:
categories = []

In [22]:
for cat in category_elements:
    name = cat.get_text(strip=True)
    link = url + cat['href']
    categories.append({"name": name, "url": link})

## Categories

In [25]:
half = len(categories) // 2 + len(categories) % 2
left = categories[:half]
right = categories[half:]
for i, (left_cat, right_cat) in enumerate(zip_longest(left, right), start=1):
    left_str = f"{i}. {left_cat['name']}" if left_cat else ""
    right_str = f"{i + half}. {right_cat['name']}" if right_cat else ""
    print(f"{left_str:<35} {right_str}")

1. Travel                           26. Autobiography
2. Mystery                          27. Parenting
3. Historical Fiction               28. Adult Fiction
4. Sequential Art                   29. Humor
5. Classics                         30. Horror
6. Philosophy                       31. History
7. Romance                          32. Food and Drink
8. Womens Fiction                   33. Christian Fiction
9. Fiction                          34. Business
10. Childrens                       35. Biography
11. Religion                        36. Thriller
12. Nonfiction                      37. Contemporary
13. Music                           38. Spirituality
14. Default                         39. Academic
15. Science Fiction                 40. Self Help
16. Sports and Games                41. Historical
17. Add a comment                   42. Christian
18. Fantasy                         43. Suspense
19. New Adult                       44. Short Stories
20. Young Adult                

## User Selects Category From List

In [28]:
choice = int(input("\nEnter the number of the category you'd like to explore: "))
selected_category = categories[choice - 1]



Enter the number of the category you'd like to explore:  24


## Scrape Books based on Category Selection

Scrape books and collect book titles

In [31]:
cat_response = requests.get(selected_category["url"])
cat_soup = BeautifulSoup(cat_response.text, 'html.parser')

book_elements = cat_soup.select('article.product_pod')
titles = set() 
books = []

for book in book_elements:
    title = book.h3.a['title']
    if title not in titles:
        titles.add(title)
        books.append({"title": title})

In [33]:
print(f"\nBooks in {selected_category['name']}:")
for i, book in enumerate(books, start=1):
    print(f"{i}. {book['title']}")


Books in Art:
1. Wall and Piece
2. Feathers: Displays of Brilliant Plumage
3. Art and Fear: Observations on the Perils (and Rewards) of Artmaking
4. The New Drawing on the Right Side of the Brain
5. History of Beauty
6. The Story of Art
7. The Art Book
8. Ways of Seeing


## Client Adding To Their Library 

In [36]:
my_library = Library()

In [38]:
while True:
    user_input = input("\nEnter the corresponding number of the book you'd like to add: ")
    if user_input.isdigit():
        selection = int(user_input)
        if 1 <= selection <= len(books):  # Optional: check it's within range
            break
        else:
            print(f"Please enter a number between 1 and {len(books)}.")
    else:
        print("Invalid input. Please enter a number.")

selected_title = books[selection - 1]["title"]


Enter the corresponding number of the book you'd like to add:  1


In [40]:
author_input = input(f"Who is the author of '{selected_title}'? ")

Who is the author of 'Wall and Piece'?  Jim Shmidt


In [42]:
new_book = Book(title=selected_title, author=author_input)
my_library.add_book(new_book)

Added Wall and Piece to my library


In [44]:
email_json = my_library.to_json()
print(email_json)

[
  {
    "Title": "Wall and Piece",
    "Author": "Jim Shmidt",
    "Available": "Yes"
  }
]


## Log In To Gmail and send the secret subject line to add the book to the library.

In [47]:
smtp_object = smtplib.SMTP('smtp.gmail.com',587)

In [49]:
smtp_object.ehlo()

(250,
 b'smtp.gmail.com at your service, [71.59.85.84]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [51]:
smtp_object.starttls()

(220, b'2.0.0 Ready to start TLS')

### Enter You Email and Generated App Password

In [54]:
email = os.getenv("GOOGLE_EMAIL")
password = os.getenv("GOOGLE_APP_PASSWORD")
smtp_object.login(email,password)

(235, b'2.7.0 Accepted')

In [60]:
from_address = email
to_address = os.getenv("EMAIL_TO")
subject = "libadd"
message = f"Subject: {subject}\n\n{email_json}"


result = smtp_object.sendmail(from_address, to_address, message)

if result == {}:
    print("Email sent successfully!")
else:
    print("Failed to send email to the following recipient(s):")
    print(result)

Email sent successfully!


## Working with the Other Classes

In [57]:
lib = Library()
lib.add_book(Book("Clean Code", "Robert C. Martin"))
lib.add_book(Magazine("Surf Today", "Kelly Slater", available=False))
lib.add_book(Newspaper("The Times", "Editorial Board"))

Added Clean Code to my library
Added Surf Today to my library
Added The Times to my library


In [59]:
print(lib.list_all())

📚 Library:
[Book] Title: Clean Code
Author: Robert C. Martin
Available: Yes
[Magazine] Title: Surf Today
Author: Kelly Slater
Available: No
[Newspaper] Title: The Times
Author: Editorial Board
Available: Yes


In [61]:
lib.remove_book("Surf Today")
print(lib.list_all())

Removed Surf Today from my library
📚 Library:
[Book] Title: Clean Code
Author: Robert C. Martin
Available: Yes
[Newspaper] Title: The Times
Author: Editorial Board
Available: Yes


In [63]:
available = sum(1 for item in lib.books if item.available)
total = len(lib.books)
print(f"{available}/{total} periodicals available")

2/2 periodicals available
